In [91]:
import numpy as np
import pandas as pd
import math

In [149]:
def get_data():
    df = pd.read_csv('Data/sample.data',header=None)
    for col in df.keys():
        df.rename(columns={col : 'atr_'+str(col)},inplace=True)
    df.rename(columns={'atr_'+str(col) : 'Label' },inplace=True)
    return df

In [150]:
def count_data(df,Attribute):
    dic = {}
    for col in Attribute:
        if col not in dic.keys(): 
            dic_2 = {}
            for atr in df[col].unique():
                dic_3 = {}
                for label in Labels: 
                    dic_3 [label]= 0
                dic_2[atr] = dic_3
            dic[col] = dic_2 

    for index, row in df.iterrows():
        for attr in Attribute:
            dic[attr][row[attr]][row['Label']] += 1
    return dic

In [151]:
def count_gain(df,dic,Attribute):
    gain = {}
    for col in Attribute:
        sum_ = 0
        for atr in df[col].unique():
            tot_row = 0
            for label in Labels:
                tot_row += dic[col][atr][label]
            tot_gain = 0
            for label in Labels:
                tmp = float(dic[col][atr][label]) / float(tot_row)
                if tmp != 0:
                    tot_gain += tmp * math.log2(tmp)
            tot_gain = tot_gain*-1
            sum_ += (tot_row/len(df.index))*tot_gain
        gain[col] = sum_
    return gain

In [152]:
def Decision_tree(df):
    Labels = set(df['Label'])
    if len(Labels) == 1:
        return Labels.pop()
    main_tree = {}
    tree = {}
    Attribute = list(df.columns)
    Attribute = Attribute[:-1]
    dic = count_data(df,Attribute)
    gain = count_gain(df,dic,Attribute)
    key_mingain = min (gain.keys(), key = (lambda k : gain[k]))
    grouped = df.groupby(df[key_mingain])
    for atr in df[key_mingain].unique():
        new_data = grouped.get_group(atr)
        del new_data[key_mingain]
        tree[atr] = Decision_tree(new_data)
    main_tree[key_mingain] = tree
    return main_tree

In [153]:
df = get_data()
Labels = set(df['Label'])
ans = Decision_tree(df)
print(ans)

{'atr_0': {'sunny': {'atr_2': {'high': 'no', 'normal': 'yes'}}, 'overcast': 'yes', 'rain': {'atr_3': {'weak': 'yes', 'strong': 'no'}}}}


In [162]:
def printDecisionTree(tree,Labels,level):
    level += " "
    for key,value in tree.items():
        if isinstance(value, dict):
            print(level+str(key)+":")
            printDecisionTree(value,Labels,level)
        else: 
            print(level+str(key)+"-->",value)

In [163]:
printDecisionTree(ans,Labels,"")

 atr_0:
  sunny:
   atr_2:
    high--> no
    normal--> yes
  overcast--> yes
  rain:
   atr_3:
    weak--> yes
    strong--> no
